# Protocolo de Modelagem

## Objetivo do Notebook
Ajustar um protocolo que teste modelos lineares e não lineares com diferentes estratégias de oversampling


## Protocolo

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.linear_model import RidgeClassifier, Perceptron
from sklearn.linear_model import LogisticRegressionCV
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.metrics import (
    accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, classification_report, confusion_matrix
)
from fairlearn.metrics import (
    MetricFrame,
    count,
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
    equal_opportunity_difference,
    equal_opportunity_ratio,
    false_positive_rate_difference,
    false_positive_rate_ratio,
    false_negative_rate_difference,
    false_negative_rate_ratio,
    selection_rate
)
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline as ImbPipeline

import os
import time
seed = 19

# Definir caminho para a pasta de resultados
results_path = "../datasets/results"

# Função para carregar datasets a partir da pasta
def carregar_dataset(caminho_csv):
    return pd.read_csv(caminho_csv)

# Função para configurar o pré-processador automaticamente
def configurar_preprocessador(x):
    categorical_features = x.select_dtypes(include=['object','category']).columns.tolist()
    numerical_features = x.select_dtypes(include=['int64', 'float64']).columns.tolist()

    # Identificar variáveis ordinais automaticamente (começam com "ordinal_")
    ordinal_features = [col for col in numerical_features if col.startswith("ordinal_")]

    # Excluir variáveis ordinais das variáveis numéricas contínuas
    numerical_continuous_features = [col for col in numerical_features if col not in ordinal_features]

    return ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_continuous_features),  # Escalar apenas variáveis contínuas
            ('cat', OneHotEncoder(drop='first'), categorical_features),  # One-Hot Encoding para categóricas
            ('ord', 'passthrough', ordinal_features)  # Manter variáveis ordinais sem transformação
        ]
    )

def treinar_e_ajustar_modelos_por_sensivel(x_train, y_train, grupo_focus_train, models, cv, smote_config, smote_label, label, arquivo):
    from imblearn.over_sampling import SMOTENC
    resultados = []
    pipelines = {}

    print(f"\nIniciando treino com oversampling pelo GRUPO_FOCUS | Cenário: {label} | Método: {smote_label}")
    print(f"Antes do oversampling - x_train shape: {x_train.shape}")

    # Se houver oversampling, criamos a base estendida com target
    if smote_config is not None:
        df_balance = x_train.copy()
        df_balance["grupo_focus"] = grupo_focus_train
        df_balance["target"] = y_train

        X = df_balance.drop(columns=["grupo_focus"])
        y = df_balance["grupo_focus"]

        categorical_features_idx = [
            i for i, col in enumerate(X.columns)
            if X[col].dtype == 'object' or X[col].dtype == 'category' or col == 'target'
        ]

        method = smote_config.pop("method", "smotenc")  # default

        if method == "random":
            from imblearn.over_sampling import RandomOverSampler
            sampler = RandomOverSampler(sampling_strategy=smote_config.get("sampling_strategy", 'auto'), random_state=seed)

        elif method == "smote_tomek":
            from imblearn.combine import SMOTETomek
            smote = SMOTENC(categorical_features=categorical_features_idx, random_state=seed)
            sampler = SMOTETomek(smote=smote, sampling_strategy=smote_config.get("sampling_strategy", 'auto'))

        elif method == "smote_enn":
            from imblearn.combine import SMOTEENN
            smote = SMOTENC(categorical_features=categorical_features_idx, random_state=seed)
            sampler = SMOTEENN(smote=smote, sampling_strategy=smote_config.get("sampling_strategy", 'auto'))

        else:  # padrão: SMOTENC
            from imblearn.over_sampling import SMOTENC
            sampler = SMOTENC(categorical_features=categorical_features_idx, random_state=seed, **smote_config)
  
        X_resampled, y_resampled = sampler.fit_resample(X, y)

        target_resampled = X_resampled["target"]
        X_resampled = X_resampled.drop(columns=["target"])
        #X_resampled["grupo_focus"] = y_resampled
    else:
        # Sem oversampling, usar os dados originais
        X_resampled = x_train.copy()
        target_resampled = y_train.copy()

    print(f"Após oversampling - X shape: {X_resampled.shape}")

    # Salvar a base de treino com target e sensitive
    df_train = X_resampled.copy()
    df_train["target"] = target_resampled
    base_filename = f"{arquivo}_{smote_label}_{label.replace(' ', '_')}_train_base_balance_GRUPO_FOCUS.csv"
    df_train.to_csv(os.path.join(results_path, base_filename), index=False)

    # Aplicar pré-processamento fora da pipeline apenas para ver o que vai acontecer com o dataset
    # etapa redundante pois o pre processamento 'oficial' esta sendo utilizado dentro da pipeline
    print("Aplicando pré-processamento nos dados de treino")
    preprocessor = configurar_preprocessador(X_resampled)
    X_transf = preprocessor.fit_transform(X_resampled)
    print(f"Shape após pré-processamento: {X_transf.shape}")

    # Treinar modelos
    for nome, config in models.items():
        print(f"Fit do modelo: {nome}", "início:", datetime.now().strftime("%H:%M:%S"))
        
        steps = [
            ('preprocessor', preprocessor), #todo dataset q entrar aqui sofrera o preprocessamento
            ('classifier', config['model'])
        ]
        pipeline = ImbPipeline(steps=steps)
        grid_search = GridSearchCV(pipeline, param_grid=config['params'], cv=cv, scoring='f1_macro')
        grid_search.fit(X_resampled, target_resampled)

        pipelines[nome] = grid_search.best_estimator_
        resultados.append({
            "modelo": nome,
            "smote": smote_label,
            "cenario": label,
            "melhor_f1": grid_search.best_score_,
            "melhores_parametros": grid_search.best_params_
        })

    return pd.DataFrame(resultados), pipelines


# NOVA Função para calcular a importância das variáveis usando permutação
def calcular_importancia_variaveis(pipeline, modelo, arquivo, label, X_val, y_val):
    """
    Calcula a importância das variáveis via permutação.
    
    Parâmetros:
        - pipeline: pipeline treinado (com preprocessor + modelo)
        - modelo: nome do modelo (str)
        - arquivo: nome do dataset (str)
        - label: cenário (str)
        - X_val: base de validação original (sem pré-processar)
        - y_val: target de validação
    Retorna:
        - Lista de dicionários com importâncias
    """
    preprocessor = pipeline.named_steps['preprocessor']
    classifier = pipeline.named_steps['classifier']

    # Aplicar o pré-processamento no conjunto de validação
    X_val_transf = preprocessor.transform(X_val)
    feature_names = preprocessor.get_feature_names_out()

    # Importância por permutação
    result = permutation_importance(
        classifier,
        X_val_transf,
        y_val,
        n_repeats=10,
        random_state=seed,
        scoring='f1_macro'
    )

    return [
        {
            "dataset": arquivo,
            "cenario": label,
            "modelo": modelo,
            "variavel": name,
            "importancia": importance
        }
        for name, importance in zip(feature_names, result.importances_mean)
    ]

# Funções para calcular taxas baseadas na matriz de confusão
def true_positive_rate(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return tp / (tp + fn) if (tp + fn) > 0 else np.nan

def true_negative_rate(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return tn / (tn + fp) if (tn + fp) > 0 else np.nan

def false_positive_rate(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return fp / (fp + tn) if (fp + tn) > 0 else np.nan

def false_negative_rate(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return fn / (fn + tp) if (fn + tp) > 0 else np.nan

# Função para calcular métricas de fairness extras (agrupadas)
def extended_fairness(y_true, y_pred, sensitive_features):
    extended_fairness_metrics = {
        "demographic_parity_difference": demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_features),
        "demographic_parity_ratio": demographic_parity_ratio(y_true, y_pred, sensitive_features=sensitive_features),
        "equalized_odds_difference": equalized_odds_difference(y_true, y_pred, sensitive_features=sensitive_features),
        "equalized_odds_ratio": equalized_odds_ratio(y_true, y_pred, sensitive_features=sensitive_features),
        "equal_opportunity_difference": equal_opportunity_difference(y_true, y_pred, sensitive_features=sensitive_features),
        "equal_opportunity_ratio": equal_opportunity_ratio(y_true, y_pred, sensitive_features=sensitive_features),
        "false_positive_rate_difference": false_positive_rate_difference(y_true, y_pred, sensitive_features=sensitive_features),
        "false_positive_rate_ratio": false_positive_rate_ratio(y_true, y_pred, sensitive_features=sensitive_features),
        "false_negative_rate_difference": false_negative_rate_difference(y_true, y_pred, sensitive_features=sensitive_features),
        "false_negative_rate_ratio": false_negative_rate_ratio(y_true, y_pred, sensitive_features=sensitive_features)
    }
    return extended_fairness_metrics

# Lista de cenários de SMOTE
smote_cenarios = [
    {"label": "Original", "config": None},
    {"label": "SMOTE_expand_5x", "config": {"sampling_strategy": {1: 5000}, "k_neighbors": 3}},
    {"label": "SMOTE_default", "config": {}},
    {"label": "RandomOverSampler", "config": {"method": "random"}},
]

# Função principal para rodar o protocolo com comparação de cenários
def executar_protocolo_com_cenarios(caminho_pasta):
    arquivos = [f for f in os.listdir(caminho_pasta) if f.endswith('.csv')]
    models = {
        'Logistic Regression': {
        'model': LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000, random_state=seed),
        'params': {'classifier__C': [1.0]}  # Apenas para compatibilidade com GridSearch
    },
    'Ridge (L2)': {
        'model': LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, random_state=seed),
        'params': {'classifier__C': [0.1, 1, 10]}  # Inverso da regularização
    },
    'Lasso (L1)': {
        'model': LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000, random_state=seed),
        'params': {'classifier__C': [0.1, 1, 10]}
    },
    'ElasticNet': {
        'model': LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, max_iter=1000, random_state=seed),
        'params': {
            'classifier__C': [0.1, 1, 10],
            'classifier__l1_ratio': [0.2, 0.5, 0.8]
        }
    },
    'SGDClassifier (Logistic)': {
        'model': SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, random_state=seed),
        'params': {'classifier__alpha': [0.0001, 0.001, 0.01]}  # alpha = 1/C
    },
    'Single Layer Perceptron': {
        'model': Perceptron(max_iter=1000, random_state=seed),
        'params': {'classifier__penalty': [None, 'l2', 'l1', 'elasticnet'],
                   'classifier__alpha': [0.0001, 0.001, 0.01]}
    },

    'SVM (RBF)': {
        'model': SVC(kernel='rbf', probability=True, random_state=seed),
        'params': {'classifier__C': [0.1, 1, 10]}
        },
    'XGBoost': {
        'model': XGBClassifier(eval_metric='logloss', random_state=seed),
        'params': {
            'classifier__learning_rate': [0.01, 0.05, 0.1],
            'classifier__max_depth': [3, 5, 7],
            'classifier__n_estimators': [50, 100, 200],
            'classifier__subsample': [0.6, 0.8, 1.0],
            'classifier__colsample_bytree': [0.6, 0.8, 1.0]
            }
        }
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    for smote_cenario in smote_cenarios:
        smote_label = smote_cenario["label"]
        smote_config = smote_cenario["config"]
        print(f"\n Rodando protocolo para SMOTE: {smote_label}")

        for arquivo in arquivos:
            print(f"\nRodando protocolo para o arquivo: {arquivo}")
            inicio_arquivo = time.time()

            # Carregar o dataset
            dataset = carregar_dataset(os.path.join(caminho_pasta, arquivo))
            
            # Definir colunas
            target_column = 'target' 
            sensitive_col_name = 'sensitive_sexo'
            print("Colunas identificadas como sensíveis:", sensitive_col_name)
            feature_columns = [col for col in dataset.columns if col not in [target_column]]

            # Separar target, features
            x = dataset[feature_columns]
            y = dataset[target_column]

            # Cenário 1: Com variáveis sensíveis
            x_with_protected = x.copy()

            # Cenário 2: Sem variáveis sensíveis
            #x_without_protected = x.drop(columns=sensitive_col_name)

            # Divisão entre treino e teste para ambos os cenários
            cenarios = [
                (x_with_protected, "Com Variáveis Sensíveis")
            ]

            resultados_completos = []
            parametros_completos = []
            importancia_variaveis = []
            fairness_results = []
            extended_fairness_results = []

            # Loop pelos cenários
            for x_data, label in cenarios:
                print(f"**Cenário: {label} | {smote_label}")

                grupo_focus = (
                    (dataset['target'] == 1) &
                    (dataset['sensitive_sexo'] == 'female') &
                    (dataset['MARRIAGE'] == 'single') &
                    (dataset['EDUCATION'] == 'high_school')).astype(int)

                x_train, x_test, y_train, y_test, grupo_focus_train, grupo_focus_test = train_test_split(x_data, y, grupo_focus, test_size=0.3, stratify=y, random_state=seed)
                
                resultados, pipelines = treinar_e_ajustar_modelos_por_sensivel(x_train, y_train, grupo_focus_train, models, cv, smote_config, smote_label, label, arquivo)
                
                for modelo, pipeline in pipelines.items():
                    print('Modelo:', modelo)
                    y_pred_test = pipeline.predict(x_test)

                    # Fairness para cada variável sensível
                    sensitive_features = dataset[sensitive_col_name].iloc[x_test.index]
                
                    metrics = {
                            'count': count,
                            'accuracy': accuracy_score,
                            'recall': recall_score,
                            'precision': precision_score,
                            'f1': f1_score,
                            'confusion_matrix':confusion_matrix,
                            'true_positive_rate': true_positive_rate,
                            'true_negative_rate': true_negative_rate,
                            'false_positive_rate': false_positive_rate,
                            'false_negative_rate': false_negative_rate,
                            'selection_rate':selection_rate}
                    
                    # Calcular métricas para cada variável sensível separadamente
                    
                    metric_frame = MetricFrame(
                        metrics=metrics,
                        y_true=y_test,
                        y_pred=y_pred_test,
                        sensitive_features=sensitive_features)
                    
                    for group in metric_frame.by_group.index:
                        row = { "dataset": arquivo, 
                            "cenario": label,
                            "smote": smote_label,
                            "modelo": modelo, 
                            "feature": sensitive_col_name,
                            "group": group
                        }
                        # Adicionar todas as métricas calculadas para o grupo
                        for metric, values in metric_frame.by_group.items():
                            row[metric] = values[group]
                        fairness_results.append(row)
                        
                    # Métricas agrupasdas de fearness
                    extended_fairness_metrics = extended_fairness(y_test, y_pred_test, sensitive_features)
                    extended_fairness_results.append({"dataset": arquivo, "cenario": label, "smote": smote_label, "modelo": modelo, "feature": sensitive_col_name, **extended_fairness_metrics})

                    # Resultados dos modelos 
                    try:
                        y_prob = pipeline.predict_proba(x_test)[:, 1]
                        auc = roc_auc_score(y_test, y_prob)
                    except AttributeError:
                        auc = None

                    resultados_completos.append({
                        "modelo": modelo,
                        "cenario": label,
                        "smote": smote_label,
                        "acuracia_teste": accuracy_score(y_test, y_pred_test),
                        "recall_teste": recall_score(y_test, y_pred_test),
                        "precisao_teste": precision_score(y_test, y_pred_test),
                        "f1_teste": f1_score(y_test, y_pred_test),
                        "auc_teste": auc,
                        "classification_report_teste": classification_report(y_test, y_pred_test, output_dict=True),
                        "confusion_matrix_teste":confusion_matrix(y_test, y_pred_test, labels=[0, 1])
                    })

                    #Melhores parametros selecionados
                    parametros_completos.append({
                        "modelo": modelo,
                        "cenario": label,
                        "smote": smote_label,
                        "melhores_parametros": pipelines[modelo].named_steps['classifier'].get_params()
                    })

                    # Calcular e armazenar importância das variáveis
                    importancia_variaveis.extend(calcular_importancia_variaveis(pipeline, modelo, arquivo, f"{label} | {smote_label}", X_val=x_test, y_val=y_test))
                    
            # Salvar resultados por dataset
            pd.DataFrame(resultados_completos).to_csv(os.path.join(results_path, f"{arquivo}_{smote_label}_resultados_completos.csv"), index=False)
            pd.DataFrame(parametros_completos).to_csv(os.path.join(results_path, f"{arquivo}_{smote_label}_parametros_completos.csv"), index=False)
            pd.DataFrame(importancia_variaveis).to_csv(os.path.join(results_path, f"{arquivo}_{smote_label}_importancia_variaveis.csv"), index=False)
            pd.DataFrame(fairness_results).to_csv(os.path.join(results_path, f"{arquivo}_{smote_label}_fairness_results.csv"), index=False)
            pd.DataFrame(extended_fairness_results).to_csv(os.path.join(results_path, f"{arquivo}_{smote_label}_extended_fairness_results.csv"), index=False)

            fim_arquivo = time.time()
            tempo_gasto = fim_arquivo - inicio_arquivo
            print(f"Tempo gasto no arquivo '{arquivo}': {tempo_gasto:.2f} segundos")

        print("\nProtocolo concluído com sucesso!")

# Executar o protocolo na pasta
executar_protocolo_com_cenarios("../datasets/processed")